In [34]:
#Imports
import numpy as np
import pandas as pd
from scipy.stats import uniform
import random
import matplotlib.pyplot as plt

In [3]:
#Variables
#Number of samples 
n = 10
#Number of loci
m = 10
#Variance of effect
var_g = 1
#Variance environment
var_e = 1

In [31]:
#Function set up 
#Function to generate genotype
def simulate_genotype(n,m):
    G= []
    f_i = list(np.random.uniform(0,1,m))
    for x in range(0,m):
        f_1 = f_i[x]
        f_0 = 1 - f_i[x]
        row = (np.random.choice([1, 0], n, p = [f_1, f_0]))
        newrow = row.tolist()
        G.append(newrow)
    #Transpose Matrix    
    finalG = np.array(G)
    np.transpose(G)
    
    assert len(G) == n, "Error in simulate_genotype"
    assert len(G[0]) == m, "Error in simulate_genotype"
    return G


#Function to generate phenotype
# Y_j = (b_i * G_ij) + e_j
def simulate_phenotype(G, var_g, var_e):
    #Select causal SNP and it's b_i (effect)
    random_SNP = random.randint(0, len(G[0])-1) #Causal SNP
    b_i = np.random.normal(0, var_g)
    Y_j = []
    for i in range(0, len(G)):
        e_j = np.random.normal(0, var_e) #Environmental variance
        Y_j.append(b_i*G[i][random_SNP] + e_j)
    return Y_j

def return_heritability(var_g, var_e):
    return var_g/(var_g + var_e)

In [38]:
#Function to check if simulated genotype follows uniform distribution
def simulated_genotype_check(G):
    for g in G:
        f_1 = sum(g)
        probf = f_1 / len(g)
simulated_genotype_check(G)        

In [32]:
#Test Functions
print("Genotype")
G = simulate_genotype(n,m)
print(G)
print("Phenotype")
Y = simulate_phenotype(G, var_g, var_e)
print(Y)
print("Heritablity")
heritability = return_heritability(var_g, var_e)
print(heritability)

Genotype
[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 1, 0, 0, 1, 0], [1, 1, 0, 0, 1, 0, 0, 0, 0, 1], [1, 0, 1, 1, 1, 1, 1, 1, 1, 1], [1, 0, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 1, 1, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 1, 1, 0], [1, 1, 1, 1, 0, 1, 0, 1, 1, 1], [1, 1, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 1, 1]]
Phenotype
[1.7609958122381202, -0.4221888971023841, 0.07608691189692457, -0.4403222951312129, -0.25276879830424065, 2.420243709143701, -0.44109617688164904, -0.4372633454809699, 0.032170799388373555, -0.7566638909870778]
Heritablity
0.5


array([[ 1,  4,  5, 12],
       [-5,  8,  9,  0],
       [-6,  7, 11, 19]])